In [2]:
import pandas as pd
import numpy as np
ruta='/content/drive/MyDrive/Colab Notebooks/ITESO/Ingeniería de características/P2022_MCD3735A/Módulo 4 - Codificación de variables categóricas/'

data=pd.read_csv(ruta+'temps.txt',sep='\s+',header=None) # Fuente: http://astro.iam.udg.mx/Estacion/NOAAPRMO.TXT
                                                         #         http://astro.iam.udg.mx/Estacion/NOAAMO.TXT
data.columns=['Temperatura máxima','Temperatura mínima','HDD','CDD','Rain']
d=['Sábado','Domingo','Lunes','Martes','Miércoles','Jueves','Viernes']
dias=[d[i%7] for i in range(len(data))]
catego=['Soleado','Mayormente soleado','Parcialmente nublado','Mayormente nublado','Nublado','Lluvioso']
prono=[catego[-1] if data['Rain'][i]>0 else catego[int(round(1.2*(data['HDD'][i]-data['CDD'][i])+2))] for i in range(len(data))]
df=pd.DataFrame(columns=['Días','Pronóstico','Temperatura máxima','Temperatura mínima'],index=range(len(data)))
df.iloc[:,2:4]=data.iloc[:,0:2]
df['Días']=dias
df['Pronóstico']=prono

In [3]:
df['Pronóstico'].unique()

array(['Parcialmente nublado', 'Mayormente nublado', 'Mayormente soleado',
       'Soleado', 'Lluvioso', 'Nublado'], dtype=object)

In [5]:
conteo=df['Pronóstico'].value_counts().to_dict()
conteo

{'Lluvioso': 2,
 'Mayormente nublado': 6,
 'Mayormente soleado': 14,
 'Nublado': 2,
 'Parcialmente nublado': 18,
 'Soleado': 6}

In [6]:
dfC=df.copy()
dfC['Pronóstico_Conteo']=df['Pronóstico'].map(conteo)
dfC.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,Pronóstico_Conteo
0,Sábado,Parcialmente nublado,28.2,11.2,18
1,Domingo,Parcialmente nublado,29.1,10.7,18
2,Lunes,Mayormente nublado,26.1,10.4,6
3,Martes,Mayormente soleado,29.4,11.1,14
4,Miércoles,Mayormente soleado,29.8,11.1,14


In [7]:
frec=((df['Pronóstico'].value_counts())/len(df['Pronóstico'])).to_dict()
frec

{'Lluvioso': 0.041666666666666664,
 'Mayormente nublado': 0.125,
 'Mayormente soleado': 0.2916666666666667,
 'Nublado': 0.041666666666666664,
 'Parcialmente nublado': 0.375,
 'Soleado': 0.125}

In [8]:
dfC['Pronóstico_Frec']=df['Pronóstico'].map(frec)
dfC.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,Pronóstico_Conteo,Pronóstico_Frec
0,Sábado,Parcialmente nublado,28.2,11.2,18,0.375000
1,Domingo,Parcialmente nublado,29.1,10.7,18,0.375000
2,Lunes,Mayormente nublado,26.1,10.4,6,0.125000
3,Martes,Mayormente soleado,29.4,11.1,14,0.291667
4,Miércoles,Mayormente soleado,29.8,11.1,14,0.291667


# Bag of words

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
count=CountVectorizer()

In [12]:
txt1="Pedro Perez Pereira, pobre pintor portuguez, pinta preciosos paisajes para poder partir para Paris."
txt2="Preciosos gatos y perros con mancha quiero. Partir de izquierda a derecha para poder llegar a algun lugar."
txt3="En un lugar de la Mancha, cuyo nombre no quiero acordarme..."

In [14]:
m=count.fit_transform([txt1,txt2,txt3])
m.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 1, 1, 0],
       [1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1]])

In [17]:
count.get_feature_names_out()

array(['acordarme', 'algun', 'con', 'cuyo', 'de', 'derecha', 'en',
       'gatos', 'izquierda', 'la', 'llegar', 'lugar', 'mancha', 'no',
       'nombre', 'paisajes', 'para', 'paris', 'partir', 'pedro',
       'pereira', 'perez', 'perros', 'pinta', 'pintor', 'pobre', 'poder',
       'portuguez', 'preciosos', 'quiero', 'un'], dtype=object)

In [19]:
pd.DataFrame(columns=count.get_feature_names_out(),data=m.toarray(),index=['txt1','txt2','txt3'])

,acordarme,algun,con,cuyo,de,derecha,en,gatos,izquierda,la,llegar,lugar,mancha,no,nombre,paisajes,para,paris,partir,pedro,pereira,perez,perros,pinta,pintor,pobre,poder,portuguez,preciosos,quiero,un
txt1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,1,1,1,1,0,1,1,1,1,1,1,0,0
txt2,0,1,1,0,1,1,0,1,1,0,1,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,0
txt3,1,0,0,1,1,0,1,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [22]:
ngrams=CountVectorizer(analyzer='word',ngram_range=(1,2))
m=ngrams.fit_transform([txt1,txt2,txt3])
pd.DataFrame(index=['txt1','txt2','txt3'],columns=ngrams.get_feature_names_out(),data=m.toarray())

,acordarme,algun,algun lugar,con,con mancha,cuyo,cuyo nombre,de,de izquierda,de la,derecha,derecha para,en,en un,gatos,gatos perros,izquierda,izquierda derecha,la,la mancha,llegar,llegar algun,lugar,lugar de,mancha,mancha cuyo,mancha quiero,no,no quiero,nombre,nombre no,paisajes,paisajes para,para,para paris,para poder,paris,partir,partir de,partir para,pedro,pedro perez,pereira,pereira pobre,perez,perez pereira,perros,perros con,pinta,pinta preciosos,pintor,pintor portuguez,pobre,pobre pintor,poder,poder llegar,poder partir,portuguez,portuguez pinta,preciosos,preciosos gatos,preciosos paisajes,quiero,quiero acordarme,quiero partir,un,un lugar
txt1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,1,1,1,1,0,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0
txt2,0,1,1,1,1,0,0,1,1,0,1,1,0,0,1,1,1,1,0,0,1,1,1,0,1,0,1,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,1,0,1,0,1,0,0
txt3,1,0,0,0,0,1,1,1,0,1,0,0,1,1,0,0,0,0,1,1,0,0,1,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1


In [23]:
_.shape #n=67

(3, 67)

La codificación correspondiente a lcada variable es la fila de dimensión $n$.